In [ ]:
#ABBILDUNGEN UND ANALYSE (TECHNISCHE POTENZIALANALYSE)
#1. Stopp, Fahren, Lenkpause Muster
#2. Batteriestand über Tage hinweg für jedes Fahrzeug
#2.1. Funktion erstellen
#2.2. Abbildung erstellen je nach Szenario
#2.3. Einzelne Tage veranschaulichen

In [ ]:
#PACKAGES AND PATHS

#!pip install pandas matplotlib
#!pip install openrouteservice
import pandas as pd
import numpy as np
import six
import matplotlib.pyplot as plt
from datetime import datetime
import matplotlib.patches as mpatches
import matplotlib.dates as mdates
import openrouteservice
from openrouteservice import convert
import requests
import json
import re
import os
import pickle
import math
import matplotlib.cm as cm
api_key = '5b3ce3597851110001cf6248dec4c079cd1b468f979908580f22cc12' #muss ich immer wieder neu erstellen über account: https://openrouteservice.org/dev/#/home
client = openrouteservice.Client(key=api_key)

FZG = ['WT-BR 270','WT-BR 231','WT-BR240', 'WT-BR 391E ', 'FR-BR 1315', 'WT-BR 229', 'WT-BR 228', 'WT-BR 2551', 'WT-BR 608', 'WT-BR 213', 'WT-BR 225', 'WT-BR 253', 'FR-BR 1317']
new_label={'WT-BR 231': 'fzg a','WT-BR240': 'fzg b', 'WT-BR 391E ': 'fzg c', 'FR-BR 1315': 'fzg d', 'WT-BR 270': 'fzg e', 'WT-BR 229': 'fzg f', 'WT-BR 228': 'fzg g', 'WT-BR 2551': 'fzg h', 'WT-BR 608': 'fzg i', 'WT-BR 213': 'fzg j', 'WT-BR 225': 'fzg k', 'WT-BR 253': 'fzg l', 'FR-BR 1317': 'fzg m'}

# datei öffnen
path = 'C:\\Users\\t.dolinga\\Documents\\Bachelorarbeit\\Daten_komplett_1908.xlsx' # Daten
save_path= 'C:\\Users\\t.dolinga\\Documents\\Bachelorarbeit\\Ergebnisse\\' #Speicherort

#DATEIEN ÖFFNEN
with open(f'{save_path}dfs_big.pkl', 'rb') as file:   
    dfs_big = pickle.load(file)
with open(f'{save_path}dfs_small.pkl', 'rb') as file:   
    dfs_small = pickle.load(file)
df_kennzahlen = pd.read_excel(f'{save_path}kennzahlen.xlsx')
# dfs_big
with open(f'{save_path}matrix_adressen.pkl', 'rb') as file:   
    matrix_adressen = pickle.load(file)

In [ ]:
#1. Stopp, FAhren, Lenkpausen Muster
def to_hours(dt):
    return dt.hour + dt.minute / 60 + dt.second / 3600

# Define colors for activities
activity_colors = {'Stopp': 'yellow', 'Fahren': 'green', 'Lenkpause': 'red'}
    
for fzg in range(len(FZG)): #für jedes Fahrzeug
    dfs = dfs_small[FZG[fzg]]
    for key, df in dfs.items():
        df['lang'] = (df['End Time'] - df['Start Time']).dt.total_seconds() / 3600
        df['start_hours'] = df['Start Time'].dt.hour + df['Start Time'].dt.minute / 60 + df['Start Time'].dt.second / 3600
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Plot each dataframe
    for i, (key, df) in enumerate(dfs.items()):
        for index, row in df.iterrows():
            ax.barh(i, row['lang'], left=row['start_hours'], color=activity_colors.get(row['Aktivität'], 'grey'))
    
    # Set labels and limits
    ax.set_xlabel('Zeit (Stunden)')
    ax.set_ylabel('Tag')
    ax.set_yticks(range(len(dfs)))
    ax.set_yticklabels([f'Tag {i+1}' for i in range(len(dfs))])
    ax.set_xlim(0, 24)
    Fahrzeug=new_label[FZG[fzg]]
    ax.set_title(Fahrzeug, fontsize=18, fontweight='bold')
    
    # Create legend
    patches = [mpatches.Patch(color=color, label=activity) for activity, color in activity_colors.items()]
    ax.legend(handles=patches, title='Aktivitäten')

    # Check if the file already exists
    file_path = f'{save_path}{new_label[FZG[fzg]]}.png'
    if os.path.exists(file_path):
        os.remove(file_path)  # Delete the previous version of the file

    # Save the new plot
    plt.savefig(file_path)
    plt.show()
    plt.clf()

In [ ]:
#2. Batteriestand über Tage hinweg für jedes Fahrzeug
#2.1. Funktion erstellen

def Verbraucht(bat_kapa, Szen_Lade, Szen_T):
    batteriekapazität=bat_kapa 
    verbrauch = 1.1 #1.1 kWh pro km 
    Faktor_Ladung = 0.000046 #4,6% pro t Ladung (0,0046% pro kg = 0,000046 pro kg) erhöht sich verbrauch, bei fracht_mean gibt es keinen Effekt
    Faktor_Steigung_mittel_hoch = 1.57 #pro Meter Elevation erhöht/verringert sich verbrauch
    Faktor_Steigung_mittel_runter = 0.51
    Temp_Szenario = Szen_T # 0= nein, 1=ja
    Temperatur = 0.75
    Temperatur_laden = 0.83
    Ladegeschwindigkeit_lp =1.667 #Szenario: 100kW =1.667 kW/min
    Ladegeschwindigkeit_stopp=0.3333   #Szenario 1: 20kW (0,3333 kW /min)
    Ladegeschwindigekti_nacht =0
    Laden_zeitverlust =10 #min die von Pausenzeit abgezogen werden, weil da noch nicht lädt/Vorbereitungszeit
    if Szen_Lade==1:
        laden_in_lenkpausen = 1 #0 ist nein, 1 ist ja
        laden_bei_stops = 1 #0 ist nein, 1 ist ja
    elif Szen_Lade==2:
        laden_in_lenkpausen = 0 #0 ist nein, 1 ist ja
        laden_bei_stops = 0 #0 ist nein, 1 ist ja
    rows, cols = len(FZG), 10
    bat_check = [] #ür jedes Fahrzeug und Tag wird 0 oder 1 abgespeichert: 0 heißt Batterie hat gereicht, 1 heißt Batterie hat nicht gereicht
    End_bat = [] # für jedes Fahrzeug und Tag wird Endbatteriestand gespeichert
    min_value = []
    fracht_mean=df_kennzahlen['Fracht'].mean()
    fracht_mean
    laden_laderampe= {}
    laden_lp= {}
    km= {}
    laden_nacht= {}
    for _ in range(rows):
        row = [None] * cols
        bat_check.append(row)
        End_bat.append(row)
        min_value.append(row)
    if Temp_Szenario == 1:
        batteriekapazität = batteriekapazität * Temperatur
        Ladegeschwindigkeit_lp =Ladegeschwindigkeit_lp*Temperatur_laden
        Ladegeschwindigkeit_stopp= Ladegeschwindigkeit_stopp * Temperatur_laden
    for fzg in range(len(FZG)): #für jedes Fahrzeug
        df = pd.read_excel(path, sheet_name=FZG[fzg])
        tage=df.iloc[2].count() # zählt alle ausgefüllten Tage (zweite zeile)
        tage=tage-1 # da erste Spalte mit Labeln ist
        Optionen = ['Stopp', 'Fahren']
        lenk = ['Ja', 'Nein', '0']
        time_format = '%H:%M:%S' 
        dfs = {} #hier kommen alle Tage rein
        laden_laderampe[FZG[fzg]]=0
        laden_lp[FZG[fzg]]=0
        km[FZG[fzg]]=0
        laden_nacht[FZG[fzg]]=0
        for tag in range(tage): #für jeden Tage
            akt = 0 #aktionen durchnummerieren
            filter = dfs_small[FZG[fzg]][tag]
            grouped = filter.groupby(['Aktivität']).size()
            fahren_count = grouped['Fahren']
            lenkpause_count = grouped['Lenkpause'] if 'Lenkpause' in grouped else 0
            stopp_count = grouped['Stopp']
        
        #Erstellen von dataframe für Verbräuche: Verbrauch (+/-) , Länge , Gesamtverbrauch
        # Idee = erstellen von dataframe punkten (Energiestand und uhrzeit -> die können am Ende verbunden werde)

            #if laden_in_lenkpausen ==1 & laden_bei_stops == 1:# länge der verbrauchsaktionen definieren (mit oder ohne Lenkpause)
            verb = [None]* (fahren_count+lenkpause_count + stopp_count+1)
            lang = [None]* (fahren_count+lenkpause_count + stopp_count+1)
            bat = [None]* (fahren_count+lenkpause_count + stopp_count+1)
            
            #Höhenmeter pro km
            dfs_big[FZG[fzg]][tag]['Elev pro km']= dfs_big[FZG[fzg]][tag]['Elevation']/dfs_big[FZG[fzg]][tag]['Distanz']
            bat[0]=batteriekapazität
            lang[0]= dfs_big[FZG[fzg]][tag]['End Time'][0]
            verb[akt]=0.0

            for i in range(len(dfs_big[FZG[fzg]][tag]['Aktivität'])): #durch einen Tag (alle ungeraden sind Fahrten)
                dfs
                batterie=batteriekapazität
                if i % 2 != 0:
                    akt=akt+1
                    if dfs_big[FZG[fzg]][tag]['Elev pro km'][i]>10:
                        Steigung = Faktor_Steigung_mittel_hoch
                    elif dfs_big[FZG[fzg]][tag]['Elev pro km'][i]<-10:
                        Steigung = Faktor_Steigung_mittel_runter
                    else:
                        Steigung=1.0

                    lang[akt]=dfs_big[FZG[fzg]][tag]['End Time'][i]
                    if dfs_big[FZG[fzg]][tag]['Lenkpause'][i]==lenk[0]: #falls Lenkpause 
                        if laden_in_lenkpausen==1: #Falls während Lenkpause Laden
                            dauer = dfs_big[FZG[fzg]][tag]['Lenkpause lang'][i]
                            total_minutes = int(dauer.total_seconds() / 60) #konvertiert time delta zu int
                            if total_minutes>Laden_zeitverlust: #nur wenn Lenkpause länger als 10min
                                lp_laden=(Ladegeschwindigkeit_lp*(total_minutes-Laden_zeitverlust))
                                laden_lp[FZG[fzg]]=laden_lp[FZG[fzg]]+lp_laden
                                verb[akt]=((verbrauch*(1+Faktor_Ladung*(dfs_big[FZG[fzg]][tag]['Fracht'][i]-fracht_mean)))*Steigung)*dfs_big[FZG[fzg]][tag]['Distanz'][i]-lp_laden
                                km[FZG[fzg]]=km[FZG[fzg]]+dfs_big[FZG[fzg]][tag]['Distanz'][i]
                            else:
                                verb[akt]=((verbrauch*(1+Faktor_Ladung*(dfs_big[FZG[fzg]][tag]['Fracht'][i]-fracht_mean)))*Steigung)*dfs_big[FZG[fzg]][tag]['Distanz'][i]
                                km[FZG[fzg]]=km[FZG[fzg]]+dfs_big[FZG[fzg]][tag]['Distanz'][i]
                            batterie = batterie+verb[akt]
                            lang[akt]=dfs_big[FZG[fzg]][tag]['End Time'][i]
                            if i>0:
                                bat[akt]=bat[akt-1]-verb[akt]
                        else:
                            lang[akt]=dfs_big[FZG[fzg]][tag]['End Time'][i]
                            verb[akt]=((verbrauch*(1+Faktor_Ladung*(dfs_big[FZG[fzg]][tag]['Fracht'][i]-fracht_mean)))*Steigung)*dfs_big[FZG[fzg]][tag]['Distanz'][i]
                            km[FZG[fzg]]=km[FZG[fzg]]+dfs_big[FZG[fzg]][tag]['Distanz'][i]
                            if i>0:
                                bat[akt]=bat[akt-1]-verb[akt]
                    else: 
                        #Verbrauch= (grundverbrauch + Fracht Faktor*Fracht + Steigungsfaktor*Elev) *Strecke (Temp Korrektur wurde schon gemacht)
                        verb[akt]=((verbrauch*(1+Faktor_Ladung*(dfs_big[FZG[fzg]][tag]['Fracht'][i]-fracht_mean)))*Steigung)*dfs_big[FZG[fzg]][tag]['Distanz'][i]
                        km[FZG[fzg]]=km[FZG[fzg]]+dfs_big[FZG[fzg]][tag]['Distanz'][i]
                        lang[akt]=dfs_big[FZG[fzg]][tag]['End Time'][i]
                        if i>0:
                            bat[akt]=bat[akt-1]-verb[akt]

                else:
                    akt=akt+1
                    if laden_bei_stops ==1 and akt!=1 and akt != len(dfs_big[FZG[fzg]][tag]['Aktivität']):
                        dauer=dfs_big[FZG[fzg]][tag]['Duration'][i]
                        total_minutes = int(dauer.total_seconds() / 60)
                        laden_stopp=(total_minutes-Laden_zeitverlust)*Ladegeschwindigkeit_stopp
                        if bat[akt-1]+laden_stopp >= batteriekapazität:
                            laden_stopp=batteriekapazität-bat[akt-1]
                        verb[akt]=-laden_stopp
                        bat[akt]=bat[akt-1]-verb[akt]
                        laden_laderampe[FZG[fzg]]=laden_laderampe[FZG[fzg]]+laden_stopp
                    else: 
                        verb[akt]=0
                        bat[akt]=bat[akt-1]
                    lang[akt]=dfs_big[FZG[fzg]][tag]['End Time'][i]
    
                #Ist Batterie unter Minimum gegeangen?
    
                bat_list=[]
                bat_list = [value for value in bat if value is not None and pd.notna(value)]
                #bat_list = [float(value) for value in bat if value is not None]
            
                    # Get the minimum value from the filtered list (wie siehts mit dem Batteriestand aus?
                min_value[fzg][tag] = min(bat_list)
                #a=bat_list[-1]
                End_bat[fzg][tag]=bat_list[-1]

    return End_bat, batteriekapazität, min_value, bat_check

In [ ]:
#2.2. Abbildung erstellen je nach Szenario
#Szenario kombination festelgen
bat_kapa=650 #300, 450, 650
Szen_Lade= 2 #1 mit Zwischenladen, 2 ohne Zwischenladen
Szen_T=0 #0 keine Extremtemperaturen, 1: Extremtemperaturen


Ergebnis=Verbraucht(bat_kapa, Szen_Lade, Szen_T)
End_bat=Ergebnis[0]
batteriekapazität=Ergebnis[1]
min_value=Ergebnis[2]
bat_check=Ergebnis[3]
for fzg in range(len(FZG)): #für jedes Fahrzeug
    print('Fahrzeug:', FZG[fzg])
    df = pd.read_excel(path, sheet_name=FZG[fzg])
    tage=df.iloc[2].count() # zählt alle ausgefüllten Tage (zweite zeile)
    tage=tage-1 # da erste Spalte mit Labeln ist
    
    for tag in range(tage): #für jeden Tage
        zwei=batteriekapazität*0.1
        eins=batteriekapazität*0.2
        if min_value[fzg][tag] < 0:
            bat_check[fzg][tag]=3
        elif min_value[fzg][tag] < zwei:
            bat_check[fzg][tag]=2
        elif min_value[fzg][tag] < eins:
            bat_check[fzg][tag]=1
        else:
            bat_check[fzg][tag]=0

#zeigt an wieviele der Fahrten in em Szenario machbar sind
# Idee grün, wenn nie 20% unterschritten wird, gelb wenn 20%, orange wenn 10% und rot wenn 0%
# Convert the list to a numpy array for easier handling
matrix_array = np.array(bat_check, dtype=object)

# Create a colormap: 1 -> red, 0 -> green, None -> gray
color_map = {
    0: 'green',
    1: 'yellow',
    2: 'orange',
    3: 'red',
    None: 'gray'
}

# Plot the color matrix with clear separation between rows
fig, ax = plt.subplots()

# Iterate through the matrix to plot each cell
for i in range(matrix_array.shape[0]):
    for j in range(matrix_array.shape[1]):
        color = color_map[matrix_array[i, j]]
        ax.add_patch(plt.Rectangle((j, -i), 1, -1, facecolor=color, edgecolor='black'))

# Set the limits, labels, and title
ax.set_xlim(0, matrix_array.shape[1])
ax.set_ylim(-matrix_array.shape[0], 0)
#ax.set_xticks(np.arange(0.5, matrix_array.shape[1], 1))
ax.set_yticks(-np.arange(0.5, matrix_array.shape[0], 1))
ax.set_xticklabels([])
ax.set_yticklabels(new_label)
#ax.invert_yaxis()
#plt.grid(True)

# Show the plot
plt.savefig(f'{save_path}Ladeszenario{Szen_Lade}_Batteriekapazität{bat_kapa}_Temperatur{Szen_T}.png')
plt.show()


In [ ]:
#2.3. Einzelne Tage veranschaulichen
#Tag und Fahrzeug auswählen  (übernimmt Szenario von oben)
fzg=1 # 0-13
tag=1 #0-maximal 10

batteriekapazität=bat_kapa 
verbrauch = 1.1 #1.1 kWh pro km 
Faktor_Ladung = 0.000046 #4,6% pro t Ladung (0,0046% pro kg = 0,000046 pro kg) erhöht sich verbrauch, bei fracht_mean gibt es keinen Effekt
Faktor_Steigung_mittel_hoch = 1.57 #pro Meter Elevation erhöht/verringert sich verbrauch
Faktor_Steigung_mittel_runter = 0.51
Temp_Szenario = Szen_T # 0= nein, 1=ja
Temperatur = 0.75
Temperatur_laden = 0.83
Ladegeschwindigkeit_lp =1.667 #Szenario: 100kW =1.667 kW/min
Ladegeschwindigkeit_stopp=0.3333   #Szenario 1: 20kW (0,3333 kW /min)
Ladegeschwindigekti_nacht =0
Laden_zeitverlust =10 #min die von Pausenzeit abgezogen werden, weil da noch nicht lädt/Vorbereitungszeit
if Szen_Lade==1:
    laden_in_lenkpausen = 1 #0 ist nein, 1 ist ja
    laden_bei_stops = 1 #0 ist nein, 1 ist ja
elif Szen_Lade==2:
    laden_in_lenkpausen = 0 #0 ist nein, 1 ist ja
    laden_bei_stops = 0 #0 ist nein, 1 ist ja

rows, cols = len(FZG), 10
bat_check = [] #ür jedes Fahrzeug und Tag wird 0 oder 1 abgespeichert: 0 heißt Batterie hat gereicht, 1 heißt Batterie hat nicht gereicht
End_bat = [] # für jedes Fahrzeug und Tag wird Endbatteriestand gespeichert
min_value = []
fracht_mean=df_kennzahlen['Fracht'].mean()
fracht_mean
laden_laderampe= {}
laden_lp= {}
km= {}
laden_nacht= {}
for _ in range(rows):
    row = [None] * cols
    bat_check.append(row)
    End_bat.append(row)
    min_value.append(row)
if Temp_Szenario == 1:
    batteriekapazität = batteriekapazität * Temperatur
    Ladegeschwindigkeit_lp =Ladegeschwindigkeit_lp*Temperatur_laden
    Ladegeschwindigkeit_stopp= Ladegeschwindigkeit_stopp * Temperatur_laden

df = pd.read_excel(path, sheet_name=FZG[fzg])
akt = 0 #aktionen durchnummerieren
filter = dfs_small[FZG[fzg]][tag]
grouped = filter.groupby(['Aktivität']).size()
fahren_count = grouped['Fahren']
lenkpause_count = grouped['Lenkpause'] if 'Lenkpause' in grouped else 0
stopp_count = grouped['Stopp']

#Erstellen von dataframe für Verbräuche: Verbrauch (+/-) , Länge , Gesamtverbrauch
# Idee = erstellen von dataframe punkten (Energiestand und uhrzeit -> die können am Ende verbunden werde)
if Temp_Szenario == 1:
    batteriekapazität = batteriekapazität * Temperatur
else:
    print('keine Temperatur anpassung')
#if laden_in_lenkpausen ==1 & laden_bei_stops == 1:# länge der verbrauchsaktionen definieren (mit oder ohne Lenkpause)
verb = [None]* (fahren_count+lenkpause_count + stopp_count+1)
lang = [None]* (fahren_count+lenkpause_count + stopp_count+1)
bat = [None]* (fahren_count+lenkpause_count + stopp_count+1)

#Höhenmeter pro km
dfs_big[FZG[fzg]][tag]['Elev pro km']= dfs_big[FZG[fzg]][tag]['Elevation']/dfs_big[FZG[fzg]][tag]['Distanz']
bat[0]=batteriekapazität
lang[0]= dfs_big[FZG[fzg]][tag]['End Time'][0]
verb[akt]=0.0

for i in range(len(dfs_big[FZG[fzg]][tag]['Aktivität'])): #durch einen Tag (alle ungeraden sind Fahrten)
    print('Aktivität',i)
    dfs
    batterie=batteriekapazität
    if i % 2 != 0:
        akt=akt+1
        print('fahrt')
        if dfs_big[FZG[fzg]][tag]['Elev pro km'][i]>10:
            Steigung = Faktor_Steigung_mittel_hoch
        elif dfs_big[FZG[fzg]][tag]['Elev pro km'][i]<-10:
            Steigung = Faktor_Steigung_mittel_runter
        else:
            Steigung=1.0

        print(verb[akt])
        lang[akt]=dfs_big[FZG[fzg]][tag]['End Time'][i]
        if dfs_big[FZG[fzg]][tag]['Lenkpause'][i]==lenk[0]: #falls Lenkpause 
            print('Lenkpausen')
            if laden_in_lenkpausen==1: #Falls während Lenkpause Laden
                print('Laden in Lenkpause') #Potentiell Laden
                dauer = dfs_big[FZG[fzg]][tag]['Lenkpause lang'][i]
                total_minutes = int(dauer.total_seconds() / 60) #konvertiert time delta zu int
                if total_minutes>Laden_zeitverlust: #nur wenn Lenkpause länger als 10min
                    lp_laden=(Ladegeschwindigkeit_lp*(total_minutes-Laden_zeitverlust))
                    verb[akt]=((verbrauch*(1+Faktor_Ladung*(dfs_big[FZG[fzg]][tag]['Fracht'][i]-fracht_mean)))*Steigung)*dfs_big[FZG[fzg]][tag]['Distanz'][i]-lp_laden
                else:
                    verb[akt]=((verbrauch*(1+Faktor_Ladung*(dfs_big[FZG[fzg]][tag]['Fracht'][i]-fracht_mean)))*Steigung)*dfs_big[FZG[fzg]][tag]['Distanz'][i]
                batterie = batterie+verb[akt]
                lang[akt]=dfs_big[FZG[fzg]][tag]['End Time'][i]
                if i>0:
                    bat[akt]=bat[akt-1]-verb[akt]
                else:
                   print('first aktivity')
            else:
                print('kein Laden in Lenkpause')
                lang[akt]=dfs_big[FZG[fzg]][tag]['End Time'][i]
                verb[akt]=((verbrauch*(1+Faktor_Ladung*(dfs_big[FZG[fzg]][tag]['Fracht'][i]-fracht_mean)))*Steigung)*dfs_big[FZG[fzg]][tag]['Distanz'][i]
                if i>0:
                    bat[akt]=bat[akt-1]-verb[akt]
                else:
                    print('first aktivity')
        else: 
            print('keine Lenkpausen')
            #Verbrauch= (grundverbrauch + Fracht Faktor*Fracht + Steigungsfaktor*Elev) *Strecke (Temp Korrektur wurde schon gemacht)
            verb[akt]=((verbrauch*(1+Faktor_Ladung*(dfs_big[FZG[fzg]][tag]['Fracht'][i]-fracht_mean)))*Steigung)*dfs_big[FZG[fzg]][tag]['Distanz'][i]
            lang[akt]=dfs_big[FZG[fzg]][tag]['End Time'][i]
            if i>0:
                bat[akt]=bat[akt-1]-verb[akt]
            else:
               print('first aktivity')
    else:
        akt=akt+1
        print('stopp')
        if laden_bei_stops ==1 and akt!=1 and akt != len(dfs_big[FZG[fzg]][tag]['Aktivität']):
            dauer=dfs_big[FZG[fzg]][tag]['Duration'][i]
            total_minutes = int(dauer.total_seconds() / 60)
            laden_stopp=(total_minutes-Laden_zeitverlust)*Ladegeschwindigkeit_stopp
            if bat[akt-1]+laden_stopp >= batteriekapazität:
                laden_stopp=batteriekapazität-bat[akt-1]
            else:
                print('Batteriekapazität nicht überschritten')
            verb[akt]=-laden_stopp
            bat[akt]=bat[akt-1]-verb[akt]
        else: 
            verb[akt]=0
            bat[akt]=bat[akt-1]
        lang[akt]=dfs_big[FZG[fzg]][tag]['End Time'][i]
plt.figure(figsize=(10, 5))
plt.plot(lang, bat, marker='o', linestyle='-', color='b')

            # Format the x-axis to show hours and minutes only
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    
            # Automatically adjust the date labels
plt.gcf().autofmt_xdate()# Format the x-axis to show dates and times clearly
plt.gcf().autofmt_xdate()
                
            # Add labels and title
plt.xlabel('Zeit (Stunden)')
plt.ylabel('Batteriestand (kWh)')
Fahrzeug=new_label[FZG[fzg]]
plt.title(f'{Fahrzeug}, Tag {tag}', fontsize=18, fontweight='bold')
                
                # Show grid
plt.grid(True)
plt.savefig(f'{save_path}{Fahrzeug}tag{tag}Ladeszenario{Szen_Lade}_Batteriekapazität{bat_kapa}_Temperatur{Szen_T}.png')                
plt.show()
